[View in Colaboratory](https://colab.research.google.com/github/tifa64/Modulation-Classification/blob/master/Radio_Signal_Processing.ipynb)

In [0]:
!pip install tensorflow
!pip install keras

In [0]:
!rm -rf datalab
!rm 2016.04C.multisnr.pkl.bz2 
!wget http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
!ls
!bzip2 -d 2016.04C.multisnr.pkl.bz2
!ls

rm: cannot remove '2016.04C.multisnr.pkl.bz2': No such file or directory
--2018-05-05 16:54:02--  http://104.197.136.14/ds/2016.04/2016.04C.multisnr.pkl.bz2
Connecting to 104.197.136.14:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 111458161 (106M) [application/x-bzip2]
Saving to: ‘2016.04C.multisnr.pkl.bz2’

2016.04C.multisnr.p 100%[===================>] 106.29M  66.5MB/s    in 1.6s    

2018-05-05 16:54:04 (66.5 MB/s) - ‘2016.04C.multisnr.pkl.bz2’ saved [111458161/111458161]

2016.04C.multisnr.pkl  2016.04C.multisnr.pkl.bz2
bzip2: Output file 2016.04C.multisnr.pkl already exists.
2016.04C.multisnr.pkl  2016.04C.multisnr.pkl.bz2


In [0]:
import pickle
import numpy as np
import scipy
def f(x,c):
  return x*c

In [0]:
with open('2016.04C.multisnr.pkl', 'rb') as f:
    Xd = pickle.load(f,encoding='bytes')

mods,snrs = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [0,1])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)


# Partition the data

In [0]:
# Partition the data
#  into training and test sets of the form we can train/test on 
#  while keeping SNR and Mod labels handy for each
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples * 0.5
print(n_train)
train_idx = np.random.choice(range(0, n_examples), size=int(n_train), replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

81030.0


# Adding diffrentiation feature

In [0]:
X_train_def = np.empty([81030, 4, 128])
for i,sample in  enumerate(X_train):
    for vector in sample:
        grad = np.gradient(vector)
        sample = np.vstack((sample,grad))
    X_train_def[i] = sample
    print(X_train_def[i])
    print(X_train_def[i].shape)

In [113]:
X_train_int = np.empty([81030, 4, 128])

for i,sample in enumerate(X_train):
    for j, vector in enumerate(sample):
        if j > 1:
            for k, element in enumerate(vector):
                X_train_int[i][j][k] = integrate.quad(f, 0, 128, element)[0]
        else:
            X_train_int[i][j] = vector
                


0.010719433426856995
